# AiiDAlab Quantum ESPRESSO app

<font color="red"><b>Caution!</b></font> Deleting this job will also remove <b>all associated nodes</b>, including every calculation initiated by this job and their respective results. This action is <b>irreversible</b>.


In [ ]:
import urllib.parse as urlparse

import ipywidgets as widgets
from IPython.display import Markdown, display

from aiida import load_profile
from aiida.orm import load_node
from aiida.tools import delete_nodes

# Load AiiDA profile
load_profile()

# Parse the primary key from the Jupyter notebook URL
url = urlparse.urlsplit(jupyter_notebook_url)  # noqa F821
query = urlparse.parse_qs(url.query)
pk = int(query["pk"][0])


def display_node_details(pk):
    try:
        node = load_node(pk)
        print(f"Node ID: {node.pk}")
        print(f"Node type: {node.process_label}")
        print(f"Label: {node.label}")
        print(f"Description: {node.description}")
        print(f"Creation time: {node.ctime}")
    except Exception as e:
        print(f"Error loading node: {e!s}")
        return False
    return True


def delete_node(pk, dry_run=True):
    if dry_run:
        _, was_deleted = delete_nodes([pk], dry_run=True)
        if was_deleted:
            print(f"Dry run: Node {pk} can be deleted.")
        return

    _, was_deleted = delete_nodes([pk], dry_run=False)
    if was_deleted:
        print(f"Node {pk} deleted successfully.")


def confirm_deletion(_):
    if delete_confirmation.value.lower() in ("y", "yes"):
        delete_node(pk, dry_run=False)
    else:
        print("Deletion aborted.")


def find_linked_qeapp_jobs(root_node_pk, process_label="QeAppWorkChain"):
    """Query all linked node with process_label = QeAppWorkChain."""
    from aiida.orm import Node, QueryBuilder
    from aiida.orm.nodes.process.workflow.workchain import WorkChainNode

    qb = QueryBuilder()
    qb.append(WorkChainNode, filters={"id": root_node_pk}, tag="root")
    qb.append(Node, with_incoming="root", tag="calcjob")
    # There are seems a bug with `with_ancestors` in the QueryBuilder, so we have to use `with_incoming` instead.
    # For the moment, it's safe to use `with_incoming` since we check it very time we delete a QEApp
    qb.append(
        WorkChainNode,
        filters={"attributes.process_label": process_label},
        with_incoming="calcjob",
    )
    results = qb.all()
    if len(results) == 0:
        return None
    return results


if display_node_details(pk):
    linked_qeapp_jobs = find_linked_qeapp_jobs(pk)
    if linked_qeapp_jobs:
        warning_html = f"""
<div style='margin: 10px; padding: 10px; border: 1px solid red; border-radius: 5px; background-color: #ffcccc;'>
    <strong style='color: red;'>Critical:</strong> Unable to delete the requested node due to dependencies.
    There are <strong>{len(linked_qeapp_jobs)}</strong> QEApp jobs linked to this node. Please delete them first:
    <ul>
"""
        for node in linked_qeapp_jobs[0]:
            warning_html += f"""<a href="./delete.ipynb?pk={node.pk}" target="_blank">{node.pk}</a><br>"""
        display(widgets.HTML(value=warning_html))
    else:
        # Ask for confirmation
        nodes, _ = delete_nodes([pk], dry_run=True)
        display(
            Markdown(
                f"**YOU ARE ABOUT TO DELETE `{len(nodes)}` NODES! THIS CANNOT BE UNDONE!**"
            )
        )
        delete_confirmation = widgets.Text(
            value="",
            placeholder='Type "yes" to confirm',
            description="Confirm:",
            disabled=False,
        )
        confirm_button = widgets.Button(description="Delete node")
        confirm_button.on_click(confirm_deletion)
        display(delete_confirmation, confirm_button)
else:
    print("No valid node found for deletion.")